In [2]:
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install Bio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached bio-1.5.9-py3-none-any.whl (276 kB)
  Using cached biopython-1.81-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached mygene-3.2.2-py2.py3-none-any.whl (5.4 kB)
  Using cached gprofiler_official-1.0.0-py3-none-any.whl (9.3 kB)
  Using cached pooch-1.7.0-py3-none-any.whl (60 kB)
  Using cached biothings_client-0.3.0-py2.py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import os
import numpy as np
from Bio import Entrez

Entrez.email = 'gchiriaco@campus.unimib.it'

import xml.etree.ElementTree as ET
import os

In [3]:
data = pd.read_csv('sample_information_American_gut_microbiome.csv.txt', sep="\t", dtype=object)
data.head()

,sample_name,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_cat,age_corrected,age_years,alcohol_consumption,alcohol_frequency,...,vioscreen_zinc,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_cat,weight_change,weight_kg,weight_units,whole_eggs,whole_grain_frequency
0,10317.000001000,Not provided,false,false,"Diagnosed by a medical professional (doctor, p...",60s,64.0,64,true,Daily,...,Not provided,Never,Regularly (3-5 times/week),Not provided,NaN,Remained stable,52,kilograms,Never,Occasionally (1-2 times/week)
1,10317.000001001,Not provided,false,false,Not provided,50s,53.0,53.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,110.0,kilograms,Not provided,Not provided
2,10317.000001002,Not provided,false,false,Not provided,50s,53.0,53.0,true,Regularly (3-5 times/week),...,Not provided,Not provided,Not provided,Not provided,NaN,Not provided,56.0,kilograms,Not provided,Not provided
3,10317.000001004,Not provided,false,false,Not provided,40s,44.0,44.0,true,Rarely (a few times/month),...,Not provided,Not provided,Not provided,Not provided,NaN,Remained stable,86.0,kilograms,Not provided,Not provided
4,10317.000001008,Not provided,false,false,Not provided,60s,66.0,66.0,false,Never,...,Not provided,Not provided,Not provided,Not provided,NaN,Increased more than 10 pounds,74.0,kilograms,Not provided,Not provided


Change all the true/false values to Yes/No

In [4]:
data[data=='true']='Yes'
data[data=='false']='No'
data[data=='I do not have this condition']='No'

In [5]:
list(data.columns)

['sample_name',
 'acid_reflux',
 'acne_medication',
 'acne_medication_otc',
 'add_adhd',
 'age_cat',
 'age_corrected',
 'age_years',
 'alcohol_consumption',
 'alcohol_frequency',
 'alcohol_types',
 'alcohol_types_beercider',
 'alcohol_types_red_wine',
 'alcohol_types_sour_beers',
 'alcohol_types_spiritshard_alcohol',
 'alcohol_types_unspecified',
 'alcohol_types_white_wine',
 'allergic_to',
 'allergic_to_i_have_no_food_allergies_that_i_know_of',
 'allergic_to_other',
 'allergic_to_peanuts',
 'allergic_to_shellfish',
 'allergic_to_tree_nuts',
 'allergic_to_unspecified',
 'altitude',
 'alzheimers',
 'animal_age',
 'animal_free_text',
 'animal_gender',
 'animal_origin',
 'animal_type',
 'anonymized_name',
 'antibiotic_history',
 'appendix_removed',
 'artificial_sweeteners',
 'asd',
 'assigned_from_geo',
 'autoimmune',
 'birth_year',
 'bmi',
 'bmi_cat',
 'bmi_corrected',
 'body_habitat',
 'body_product',
 'body_site',
 'bowel_movement_frequency',
 'bowel_movement_quality',
 'breastmilk_for

Selecting only the samples from feces

In [6]:

data = data.loc[data.body_site == "UBERON:feces",:]

Removing all the subjects with special diseases and conditions

In [7]:
cond_no_disease = ( (data['alcohol_frequency']!='Daily') &
                    ((data['autoimmune']=='No') | (data['autoimmune']=='Not provided')) &
                    ((data['cancer']=='No') | (data['cancer']=='Not provided')) &
                    ((data['cdiff']=='No') | (data['cdiff']=='Not provided')) & #Clostridioides difficile
                    ((data['diabetes']=='No') | (data['diabetes']=='Not provided')) &
                    ((data['ibd']=='No') | (data['ibd']=='Not provided')) & #inflammatory bowel disease
                    ((data['ibs']=='No') | (data['ibs']=='Not provided')) & #inflamatiry bowel syndrome
                    ((data['kidney_disease']=='No') | (data['kidney_disease']=='Not provided')) &
                    ((data['liver_disease']=='No') | (data['liver_disease']=='Not provided')) &
                    ((data['mental_illness_type_anorexia_nervosa']=='No') | (data['mental_illness_type_anorexia_nervosa']=='Not provided')) &
                    ((data['mental_illness_type_bulimia_nervosa']=='No') | (data['mental_illness_type_bulimia_nervosa']=='Not provided')) &
                    ((data['sibo']=='No') | (data['sibo']=='Not provided'))) #small intestinal bacterial overgrowth

data_no_disease = data.loc[cond_no_disease, :]

Number of subjects that removed refined sugars from their diet

In [8]:
data_no_disease.specialized_diet_exclude_refined_sugars.value_counts()

Not provided    7417
No              4633
Yes              633
Unspecified       22
Name: specialized_diet_exclude_refined_sugars, dtype: int64

Selecting only Male/Females and subject with age between 20s and 60s

In [9]:
data_no_disease.age_cat.value_counts()

30s             2359
40s             2288
50s             2221
60s             1735
20s             1352
Not provided     748
child            733
70+              594
teen             382
Unspecified      287
baby               6
Name: age_cat, dtype: int64

In [10]:
data_no_disease.sex.value_counts()

female          6178
male            5976
Not provided     526
unspecified       18
other              7
Name: sex, dtype: int64

In [11]:
data_f = data_no_disease.loc[((data_no_disease['age_cat'] == '20s') |
                          (data_no_disease['age_cat'] == '30s') |
                          (data_no_disease['age_cat'] == '40s') |
                          (data_no_disease['age_cat'] == '50s')|
                          (data_no_disease['age_cat'] == '60s'))  &
                          ((data_no_disease.sex == 'female') | (data_no_disease.sex == 'male')) ,:]

In [12]:
pd.crosstab(data_f.age_cat,
            data_f.specialized_diet_exclude_refined_sugars, 
            values=data_f.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Not provided,Unspecified,Yes,All
age_cat,,,,,
20s,573.0,698.0,NaN,59.0,1330
30s,869.0,1295.0,3.0,128.0,2295
40s,817.0,1330.0,3.0,105.0,2255
50s,771.0,1284.0,6.0,101.0,2162
60s,557.0,1026.0,4.0,92.0,1679
All,3587.0,5633.0,16.0,485.0,9721


In [13]:
pd.crosstab(data_f.sex,
            data_f.specialized_diet_exclude_refined_sugars, 
            values=data_f.sample_name,
            aggfunc=len, 
            margins = True)

specialized_diet_exclude_refined_sugars,No,Not provided,Unspecified,Yes,All
sex,,,,,
female,1824,2945,12,250,5031
male,1763,2688,4,235,4690
All,3587,5633,16,485,9721


Creation of the Control/Treatment groups:
- Control: subjects without a particular health condition, that don't follow any specilized diet

- Treatment: subjects without a paritcular health condition, that exclude refined sugars from their diet and don't follow any other specilized diet

In [14]:
[x for x in list(data_f.columns) if x.lower().startswith("specialized_diet_")]

['specialized_diet_exclude_dairy',
 'specialized_diet_exclude_nightshades',
 'specialized_diet_exclude_refined_sugars',
 'specialized_diet_fodmap',
 'specialized_diet_halaal',
 'specialized_diet_i_do_not_eat_a_specialized_diet',
 'specialized_diet_kosher',
 'specialized_diet_modified_paleo_diet',
 'specialized_diet_other_restrictions_not_described_here',
 'specialized_diet_paleodiet_or_primal_diet',
 'specialized_diet_raw_food_diet',
 'specialized_diet_unspecified',
 'specialized_diet_westenprice_or_other_lowgrain_low_processed_fo',
 'specialized_diet_westenprice_or_other_lowgrain_low_processed_food_diet']

In [15]:
data_f= data_f.drop("specialized_diet_unspecified",axis=1)

Function that assigns every subject to a group

In [31]:
data_f.specialized_diet_exclude_refined_sugars.value_counts()

Not provided    5633
No              3587
Yes              485
Unspecified       16
Name: specialized_diet_exclude_refined_sugars, dtype: int64

In [32]:
def get_groups(data):
    exclude_sugar = None
    other_diets = False
    specialized_diet = False

    list_col = list(data.index)
    list_col = [x for x in list_col if x.lower().startswith("specialized_diet_")]
    for col in list_col:
        if col =="specialized_diet_i_do_not_eat_a_specialized_diet":
            if data[col] == "No":
                specialized_diet = True
        elif col == "specialized_diet_exclude_refined_sugars":
            if data[col] == "Yes":
                exclude_sugar = True
            if data[col] == "No":
                exclude_sugar = False
        else:
            if data[col] == "Yes" or data[col] == "Not provided":
                other_diets = True
    
    if specialized_diet:
        
        if exclude_sugar is not None and exclude_sugar and not other_diets:
            return "Treatment"
        else:
            return "None"
    
    else:
        if exclude_sugar is None or exclude_sugar or other_diets:
            return "None"
        
        
        return "Control"
        

In [33]:
data_f['group'] = data_f.apply(get_groups, axis=1)

Only 110 subjects have the characteristics to be in the Treatment group

In [34]:
data_f.group.value_counts()

None         7171
Control      2440
Treatment     110
Name: group, dtype: int64

Excluding all subjects that are neither in the control or treatment group

In [35]:
data_groups = data_f.loc[((data_f.group == "Control") | (data_f.group == "Treatment"))]

Sex distribution in the two groups

In [36]:
print(data_groups.groupby(["group","sex"]).sample_name.nunique())

group      sex   
Control    female    1202
           male      1238
Treatment  female      53
           male        57
Name: sample_name, dtype: int64


Sex and age distribution in the two groups

In [37]:
print(data_groups.groupby(["group","sex","age_cat"]).sample_name.nunique())

group      sex     age_cat
Control    female  20s        183
                   30s        271
                   40s        281
                   50s        289
                   60s        178
           male    20s        199
                   30s        299
                   40s        264
                   50s        263
                   60s        213
Treatment  female  20s          3
                   30s         14
                   40s         11
                   50s         12
                   60s         13
           male    20s         13
                   30s          9
                   40s         11
                   50s         12
                   60s         12
Name: sample_name, dtype: int64


In [38]:
data_groups.age_years = data_groups.age_years.astype('float16')
print(data_groups.groupby(["group","sex"]).age_years.mean())

group      sex   
Control    female    44.46875
           male      44.56250
Treatment  female    48.62500
           male      44.90625
Name: age_years, dtype: float16


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


FastQ pull:

In [43]:
def get_fastq(sample_names, download = False):
    
    sample_name = list(sample_names)
    biosample_ids = list(sample_names)
    run_ids = list(sample_names) 
    i = 0
    
    for sample in sample_name:
        i+=1
        print(f"({i})\tGetting {sample_name[i-1]} Biosample ID")
        samplehandle = Entrez.esearch(db='biosample', term=sample)
        sampleread = Entrez.read(samplehandle)
                
        if len(sampleread['IdList']) == 0:
            print("No BiosampleID found")
            biosample_ids[i-1] = "Not found"
            run_ids[i-1] = "Not found"

        else: 
            print(f"\tFound {len(sampleread['IdList'])} BiosampleID: {sampleread['IdList']}")

            for bioid in sampleread['IdList']:
                try:
                    biohandle = Entrez.efetch(db='biosample', id=bioid, retmode='xml')
                    bioxml = ET.fromstring(biohandle.read())
                    sraids = bioxml.findall('.//BioSample//Ids//Id')
                    sraids = [x.text for x in sraids if x.attrib['db']=="SRA"]
                except:
                    run_ids[i-1] = "Not found"
                    continue 
                
                for sraid in sraids:
                    srahandle = Entrez.esearch(db='sra', term=sraid,retmode="xml")
                    srareads = Entrez.read(srahandle)["IdList"]
                    print(f"\tFound {len(srareads)} SRA ID: {srareads}")

                    sraidhandles =[Entrez.efetch(db='sra', id=s) for s in srareads]
                    root = [ET.fromstring(handle.read()) for handle in sraidhandles]
                    identifier = [r.find('.//EXPERIMENT_PACKAGE//RUN_SET//RUN') for r in root]
                    instrument = [r.find('.//EXPERIMENT_PACKAGE//EXPERIMENT//PLATFORM//ILLUMINA//INSTRUMENT_MODEL').text for r in root]

                    filt_id = [identifier[k] for k in range(len(identifier)) if instrument[k] == "Illumina MiSeq"]

                    if len(filt_id)>0 and filt_id[0] is not None:
                        one_id = filt_id[0].attrib["accession"]
                        print('\tSelected 1 Run ID:', one_id)
                        biosample_ids[i-1]=bioid
                        run_ids[i-1] = one_id
                    else:
                        print('\tNo RunID found')
                        run_ids[i-1] = "Not found"

                
                    # Download FASTQ dei RUNid
                    if download:
                        if not os.path.exists(f"fastq"):
                            os.mkdir(f"fastq")

                        print('\tDownloading FASTQ...')
                        command = f'sratoolkit.3.0.5-ubuntu64/bin/fastq-dump --outdir fastq/ --readids ' + one_id
                        os.system(command)


    dict = {'sample_names': sample_name, 
                'biosample_ids': biosample_ids,
                'run_ids': run_ids}  

    return dict    

Treatment group data

In [44]:
data_treatment = data_groups.loc[(data_groups.group == "Treatment")]

Control group data (selecting only 120 subjects)

In [45]:
data_control = data_groups.loc[(data_groups.group == "Control")]
data_control = data_control.groupby(["sex","age_cat"]).head(12)

Treatment group fastq pulling

In [46]:
dict_treatment = get_fastq(data_treatment.sample_name, download = True)

(1)	Getting 10317.000023139 Biosample ID
	Found 1 BiosampleID: ['7353729']
	Found 1 SRA ID: ['4279800']
	Selected 1 Run ID: ERR2032495
Read 16572 spots for ERR2032495
Written 16572 spots for ERR2032495
(2)	Getting 10317.000027811 Biosample ID
	Found 1 BiosampleID: ['7353735']
	Found 1 SRA ID: ['4279806']
	Selected 1 Run ID: ERR2032501
Read 19562 spots for ERR2032501
Written 19562 spots for ERR2032501
(3)	Getting 10317.000033280 Biosample ID
	Found 1 BiosampleID: ['8577201']
	Found 1 SRA ID: ['5144891']
	Selected 1 Run ID: ERR2313963
Read 27488 spots for ERR2313963
Written 27488 spots for ERR2313963
(4)	Getting 10317.000038261 Biosample ID
	Found 1 BiosampleID: ['8614208']
	Found 1 SRA ID: ['5162312']
	Selected 1 Run ID: ERR2318007
Read 39992 spots for ERR2318007
Written 39992 spots for ERR2318007
(5)	Getting 10317.000041075 Biosample ID
	Found 1 BiosampleID: ['8728556']
	Found 5 SRA ID: ['14635658', '14635351', '14634860', '14631050', '5245039']
	Selected 1 Run ID: ERR2404913
Read 3426

Control group fastq pulling

In [47]:
dict_control = get_fastq(data_control.sample_name, download = True)

(1)	Getting 10317.000001166 Biosample ID
	Found 1 BiosampleID: ['7353694']
	Found 1 SRA ID: ['4279766']
	Selected 1 Run ID: ERR2032461
Read 36116 spots for ERR2032461
Written 36116 spots for ERR2032461
(2)	Getting 10317.000003100 Biosample ID
	Found 1 BiosampleID: ['14618395']
	Found 1 SRA ID: ['10591931']
	Selected 1 Run ID: ERR4018879
Read 27847 spots for ERR4018879
Written 27847 spots for ERR4018879
(3)	Getting 10317.000004936 Biosample ID
	Found 1 BiosampleID: ['7353699']
	Found 2 SRA ID: ['4279771', '5162666']
	Selected 1 Run ID: ERR2032466
Read 5 spots for ERR2032466
Written 5 spots for ERR2032466
(4)	Getting 10317.000010862 Biosample ID
	Found 1 BiosampleID: ['7353704']
	Found 1 SRA ID: ['4279776']
	Selected 1 Run ID: ERR2032471
Read 37468 spots for ERR2032471
Written 37468 spots for ERR2032471
(5)	Getting 10317.000012018 Biosample ID
	Found 1 BiosampleID: ['18048072']
	Found 1 SRA ID: ['13598743']
	Selected 1 Run ID: ERR5325492
Read 19774 spots for ERR5325492
Written 19774 spot

Id dataframe for both groups

In [48]:
treatment_fastq = pd.DataFrame.from_dict(dict_treatment)
control_fastq = pd.DataFrame.from_dict(dict_control)

Selecting only subjects for whom fastq was found

In [49]:
treatment_fastq = treatment_fastq.loc[treatment_fastq['run_ids'] != "Not found"].reset_index(drop= True)
control_fastq = control_fastq.loc[control_fastq['run_ids'] != "Not found"].reset_index(drop= True)

Selecting only 100 subjects for each group

In [50]:
data_control = data_control.loc[data_control['sample_name'].isin(control_fastq.sample_names)]
data_treatment = data_treatment.loc[data_treatment['sample_name'].isin(treatment_fastq.sample_names)]

In [51]:
data_control = data_control.groupby(["sex","age_cat"]).head(10).reset_index(drop=True)
data_treatment = data_treatment.head(100).reset_index(drop=True)

Concatenation of the two groups

In [52]:
data_final_groups = pd.concat([data_treatment,data_control]).reset_index(drop=True)

In [53]:
control_fastq = control_fastq.loc[control_fastq.sample_names.isin(data_control['sample_name'])]
treatment_fastq = treatment_fastq.loc[treatment_fastq.sample_names.isin(data_treatment['sample_name'])]

In [54]:
all_fastq = pd.concat([control_fastq,treatment_fastq]).reset_index(drop=True)

In [55]:
all_fastq

,sample_names,biosample_ids,run_ids
0,10317.000001166,7353694,ERR2032461
1,10317.000003100,14618395,ERR4018879
2,10317.000004936,7353699,ERR2032466
3,10317.000010862,7353704,ERR2032471
4,10317.000012018,18048072,ERR5325492
...,...,...,...
195,10317.000108434,18050057,ERR5328033
196,10317.000108496,18047336,ERR5328992
197,10317.000108903,18047274,ERR5329027
198,10317.000108906,18047334,ERR5329028


Saving the dataframe with all needed info

In [57]:
data_final_groups = pd.merge(data_final_groups,
                             all_fastq,
                             how = "inner",
                             left_on = "sample_name",
                             right_on="sample_names")
data_final_groups.to_csv("data_final_groups.csv")

# FASTQ - Quality check

In [58]:
from Bio import SeqIO
from os import listdir
from os.path import isfile, join
import os
import pandas as pd

In [59]:
def filter_quality(fastq_path,min_quality = 20,verbose = True):
    
    tot_reads = []
    saved_reads = []

    if not os.path.exists('good_quality_' + str(min_quality)):
        os.mkdir(f'good_quality_{str(min_quality)}')
    
    for f in os.listdir(fastq_path):
        print(f"Reading {f}")
        num_reads = 0 
        for rec in SeqIO.parse(os.path.join(fastq_path,f), "fastq"):
            num_reads +=1
            good_reads = (rec for rec in \
                            SeqIO.parse(os.path.join(fastq_path,f), "fastq") \
                                if min(rec.letter_annotations["phred_quality"]) >= min_quality)

        count = SeqIO.write(good_reads, f"good_quality_{str(min_quality)}/{f}", "fastq")
        print(f"Saved {count} reads")
        print(f"{num_reads} reads\n\n")
        tot_reads.append(num_reads)
        saved_reads.append(count)
    
    return os.listdir(fastq_path), tot_reads, saved_reads
    



In [60]:
names, tot_reads, good_reads = filter_quality("fastq")

Reading ERR2579789.fastq
Saved 822 reads
3583 reads


Reading ERR2032558.fastq
Saved 11121 reads
36816 reads


Reading ERR2318153.fastq
Saved 6909 reads
36283 reads


Reading ERR5328190.fastq
Saved 4367 reads
15288 reads


Reading ERR2032538.fastq
Saved 9420 reads
36381 reads


Reading ERR5326482.fastq
Saved 9406 reads
20175 reads


Reading ERR2404913.fastq
Saved 16881 reads
34268 reads


Reading ERR2314152.fastq
Saved 30183 reads
49733 reads


Reading ERR5332232.fastq
Saved 8168 reads
21965 reads


Reading ERR1842727.fastq
Saved 162 reads
37171 reads


Reading ERR1854687.fastq
Saved 212 reads
3249 reads


Reading ERR5328798.fastq
Saved 6696 reads
14172 reads


Reading ERR2579848.fastq
Saved 8218 reads
38207 reads


Reading ERR1842731.fastq
Saved 22 reads
18850 reads


Reading ERR2032818.fastq
Saved 7711 reads
20563 reads


Reading ERR2405327.fastq
Saved 8663 reads
17131 reads


Reading ERR2523877.fastq
Saved 7210 reads
15366 reads


Reading ERR2304018.fastq
Saved 13434 reads
27533 rea

In [61]:
good_reads_df = pd.DataFrame({"id":names,"tot_reads":tot_reads,"good_reads":good_reads})

In [62]:
good_reads_df.to_csv('good_reads.csv')

In [8]:
good_reads_df = pd.read_csv("good_reads.csv",index_col=0)

In [12]:
good_reads_df.sort_values(by="good_reads").head(15)

,id,tot_reads,good_reads
204,ERR2318150.fastq,13,1
189,ERR2032466.fastq,5,2
201,ERR2319277.fastq,34,4
114,ERR1842698.fastq,7525,7
142,ERR1842726.fastq,12478,8
34,ERR2318152.fastq,57,20
13,ERR1842731.fastq,18850,22
115,ERR1842797.fastq,28329,23
37,ERR1842877.fastq,17826,24
139,ERR1842722.fastq,40861,29
